In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [7]:
users = pd.read_csv('./data/users.csv')
reviews = pd.read_csv('./data/reviews.csv', low_memory=False)
aspects = pd.read_csv('./data/aspects.csv', low_memory=False)
features = pd.read_csv('./data/features.csv', low_memory=False)
orgs = pd.read_csv('./data/organisations.csv', low_memory=False)
rubrics = pd.read_csv('./data/rubrics.csv', low_memory=False)

* reviews - В этом файле дана информация об отзывах и оценках, оставленных некоторым множеством жителей Москвы и Санкт-Петерубрга в течение обучающего периода
* organisations - Информация об организациях
* users - Информация о городе проживания пользователя
* aspects - Описание извлекаемых из отзывов аспектов. Множество аспектов извлекается из отзыва с помощью NLP-алгоритма и может быть неточным.
* features - Описание особенностей организаций. Как правило, множество особенностей организации заполняется ее владельцем и может быть неточным.
* rubrics - Описание рубрик организаций
* test_users - Множество пользователей, для которых необходимо сделать предсказание

In [8]:
to_list = lambda rubrics: [int(rubric) for rubric in str(rubrics).split(' ')]
def apply_to_columns(df, columns, func=to_list):
    for column in columns:
        df.loc[~df[column].isnull(), column] = df.loc[~df[column].isnull(), column].apply(func)

In [9]:
cols = ['rubrics_id', 'features_id']
apply_to_columns(orgs, cols)
orgs.head()

,org_id,city,average_bill,rating,rubrics_id,features_id
0,16848414477362211020,spb,1000.0,4.479702,"[30776, 31375]","[1018, 1509, 11177, 11617, 11629, 11704, 11867..."
1,1430604733320164116,spb,1000.0,4.514509,"[30776, 30770]","[246, 1018, 11617, 11629, 11704, 11867, 20422,..."
2,9880309324224147401,spb,1000.0,3.884615,"[30770, 30774]","[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
3,5617879987171966456,spb,1000.0,NaN,"[30774, 30775]","[1018, 1509, 10596, 11177, 11629, 11634, 11704..."
4,5241461680470612149,spb,1000.0,4.532468,[30776],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."


In [10]:
reviews = reviews.merge(users, on='user_id')
reviews = reviews.rename({'city': 'user_city'}, axis=1)
reviews = reviews.merge(orgs[['org_id', 'city']], on='org_id')
reviews = reviews.rename({'city': 'org_city'}, axis=1)
reviews

,user_id,org_id,rating,ts,aspects,user_city,org_city
0,16998268288908323644,7184895086928047809,2.0,105,NaN,msk,msk
1,3121447338909258868,7184895086928047809,5.0,464,NaN,msk,msk
2,1970649778250883025,7184895086928047809,3.0,789,NaN,msk,msk
3,7554889464530643866,7184895086928047809,4.0,936,NaN,msk,msk
4,15907910894057053620,7184895086928047809,1.0,1143,NaN,msk,msk
...,...,...,...,...,...,...,...
3640830,16504916268155591133,11379950099553543298,1.0,1138,NaN,spb,spb
3640831,6729633349339708345,4127027708972853576,5.0,984,NaN,msk,msk
3640832,12811636719149152603,1870939193149876281,5.0,389,NaN,msk,msk
3640833,16479336894539955366,9457835296761142609,5.0,1068,NaN,msk,msk


In [11]:
columns = ['aspects']
apply_to_columns(reviews, columns)

## Заполним средний рейтинг заведения с помощью оценок поставленными пользователями

In [ ]:
def fill_avg_rat(row):
    avg_rat = reviews[reviews['org_id'] == row['org_id']]['rating'].mean()
    return avg_rat

In [ ]:
orgs['new_rat'] = orgs.progress_apply(fill_avg_rat, axis=1)

100%|██████████| 66405/66405 [07:38<00:00, 144.70it/s]


In [ ]:
orgs

,org_id,city,average_bill,rating,rubrics_id,features_id,new_rat
0,16848414477362211020,spb,1000.0,4.479702,"[30776, 31375]","[1018, 1509, 11177, 11617, 11629, 11704, 11867...",4.476094
1,1430604733320164116,spb,1000.0,4.514509,"[30776, 30770]","[246, 1018, 11617, 11629, 11704, 11867, 20422,...",4.550218
2,9880309324224147401,spb,1000.0,3.884615,"[30770, 30774]","[1018, 11177, 11617, 11629, 11704, 11867, 2042...",3.953125
3,5617879987171966456,spb,1000.0,NaN,"[30774, 30775]","[1018, 1509, 10596, 11177, 11629, 11634, 11704...",3.722222
4,5241461680470612149,spb,1000.0,4.532468,[30776],"[1018, 11177, 11617, 11629, 11704, 11867, 2042...",4.680000
...,...,...,...,...,...,...,...
66400,6886326702594574487,spb,1000.0,4.504259,"[30776, 30774, 30770]","[246, 1018, 1509, 11629, 11704, 20422, 21247, ...",4.485030
66401,5569256238203823632,spb,1000.0,4.519685,[30774],"[1018, 11177, 11617, 11629, 11704, 11867, 2042...",4.563739
66402,1532407218667788426,spb,1000.0,4.416576,"[30776, 31375]","[1018, 1509, 11177, 11617, 11629, 11704, 11867...",4.409263
66403,826906323277325942,spb,1000.0,4.384032,"[31375, 30776]","[1018, 1509, 11177, 11617, 11629, 11704, 11867...",4.364045


In [14]:
orgs['rat_diff'] = orgs['rating'] - orgs['new_rat']

In [12]:
reviews = reviews.merge(orgs, on='org_id', how='left')\
    .rename(columns={'rating_x': 'rating', 'rating_y':'avg_rating'})\
    .drop(['city'], axis=1)

## Возьмем данные за последние N дней

In [13]:
new_rev = reviews[reviews['ts'] >= 1217 - 428]

In [15]:
#valid_reviews = new_rev
valid_reviews = new_rev[(new_rev['rating'] >= 4.0) & (new_rev['avg_rating'] >= 4.0)]

In [16]:
valid_reviews

,user_id,org_id,rating,ts,aspects,user_city,org_city,average_bill,avg_rating,rubrics_id,features_id
3,7554889464530643866,7184895086928047809,4.0,936,NaN,msk,msk,500.0,4.038688,[30771],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
13,15024702726583868341,7184895086928047809,5.0,1022,NaN,msk,msk,500.0,4.038688,[30771],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
16,9244382993634544939,7184895086928047809,5.0,897,NaN,msk,msk,500.0,4.038688,[30771],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
30,8621017725380872971,7184895086928047809,5.0,822,NaN,msk,msk,500.0,4.038688,[30771],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
32,3630334064011961783,7184895086928047809,5.0,955,NaN,msk,msk,500.0,4.038688,[30771],"[1018, 11177, 11617, 11629, 11704, 11867, 2042..."
...,...,...,...,...,...,...,...,...,...,...,...
3640822,3433147116291244904,2404320862555012761,5.0,941,NaN,msk,msk,NaN,4.857143,[30774],"[11867, 20422, 273469383]"
3640823,9315872414048297431,4457136954359416179,5.0,1007,NaN,msk,msk,NaN,5.000000,[30771],"[1018, 11617]"
3640827,14219387695304419906,17279449842621849676,5.0,1212,NaN,msk,msk,500.0,5.000000,"[30774, 30775]","[246, 1018, 1509, 11617, 11629, 11704, 11867, ..."
3640829,15855894068288546797,17211700938371321958,5.0,945,NaN,msk,msk,NaN,5.000000,[30771],NaN


In [25]:
# набор отзывов только от туристов
tourist_reviews = valid_reviews[valid_reviews['user_city'] != valid_reviews['org_city']]

# выбираем самые популярные места среди туристов из Москвы и Питера
msk_orgs = tourist_reviews[tourist_reviews['org_city'] == 'msk']['org_id']
msk_orgs = msk_orgs.value_counts().index[:20].to_list()

spb_orgs = tourist_reviews[tourist_reviews['org_city'] == 'spb']['org_id']
spb_orgs = spb_orgs.value_counts().index[:20].to_list()

In [26]:
test = pd.read_csv('data/test_users.csv')
test['city'] = test.merge(users, on='user_id')['city']

In [27]:
msk_orgs = str(' '.join(map(str, msk_orgs)))
spb_orgs = str(' '.join(map(str, spb_orgs)))

test_users = pd.read_csv('data/test_users.csv')
test_users['city'] = test_users.merge(users, on='user_id')['city']

choose = lambda x: spb_orgs if x['city'] == 'msk' else msk_orgs
target = test_users.apply(choose, axis=1)

predictions = test_users[['user_id']]
predictions['target'] = target

predictions

,user_id,target
0,3545210947248911048,12046097390037935713 6838233943148091808 20703...
1,15271987121288045390,12046097390037935713 6838233943148091808 20703...
2,15016858616184265932,12046097390037935713 6838233943148091808 20703...
3,12457244142928722989,12046097390037935713 6838233943148091808 20703...
4,13339684649926251468,15250345250621165867 15684663803879321952 9104...
...,...,...
16962,1191875913294598364,12046097390037935713 6838233943148091808 20703...
16963,3866507700167344338,12046097390037935713 6838233943148091808 20703...
16964,11434952144484188987,12046097390037935713 6838233943148091808 20703...
16965,7010426792722803474,15250345250621165867 15684663803879321952 9104...


In [28]:
predictions.to_csv('answers.csv', index=False)

## С заполнением среднего рейтинга оценка - 4.76
## Без заполнения оценка - 4.81
## Выборка за последние 107 дней - оценка 5.02
## Выборка за последние 214 дней - оценка 5.17
## Выборка за последние 321 дня - оценка 5.18

## москвичи - москва, питербуржцы - питер
## москвичи - питер, питербуржцы - москва

## Составить новый рейтинг за последний год
## Составить рейтинг по тренировочным данным, если возрос ставят хорошие оценки, если упал, нет
## Оценка 3.8 :(